# 포탈 02 - 네이버 파이낸스 종목 부가정보

(SQLite, stock_desc 테이블)

<img src="http://i.imgur.com/grQHNKG.jpg">

#### 2017 FinanceData http://fb.com/financedata

In [1]:
import pandas as pd
from IPython.display import display

In [2]:
import sqlite3
conn = sqlite3.connect('findata.db')

In [3]:
# 테이블  읽기
df_master = pd.read_sql_query('SELECT * FROM stock_master', conn, index_col='code')
df_master.head()

,index,name,sector_code,sector,telephone,address
code,,,,,,
060310,0,3S,032602,전자부품 제조업,02-896-9474,서울특별시 금천구 시흥대로71길 30-1
095570,1,AJ네트웍스,126903,산업용 기계 및 장비 임대업,02-6240-0800,"서울특별시 송파구 정의로8길 9 (문정동,AJ빌딩)"
068400,2,AJ렌터카,126901,운송장비 임대업,1544-1600,서울특별시 구로구 서부샛길 822
006840,3,AK홀딩스,137105,"회사본부, 지주회사 및 경영컨설팅 서비스업",02-6923-2921,서울특별시 구로구 구로중앙로 152(구로동)
054620,4,APS홀딩스,032902,특수 목적용 기계 제조업,031-776-1800,경기도 화성시 동탄면 동탄산단9길 23-12 (동탄면)


# stock_desc 테이블 생성

In [4]:
# create stock_desc table
def create_stock_desc(conn):
    # stock_desc 테이블 존재 여부 확인
    exist_sql = "SELECT name FROM sqlite_master WHERE type='table' AND name='stock_desc';"
    df_exist = pd.read_sql(exist_sql, conn)
    if len(df_exist) > 0:
        return False
    
    # create stock_desc
    df_master = pd.read_sql('SELECT * FROM stock_master', conn)
    df_desc = df_master[['code', 'name']].copy()
    df_desc['hompage'] = ''
    df_desc['telephone'] = ''
    df_desc['address'] = ''
    df_desc['market'] = ''
    df_desc['wics'] = ''
    df_desc['name_en'] = ''
    df_desc['desc'] = ''
    df_desc['desc_date'] = ''
    df_desc.to_sql('stock_desc', conn)
    return True
        
create_stock_desc(conn)

False

# 태그 검색

* dt 태그를 검색 class='line-left'

In [5]:
import requests
from bs4 import BeautifulSoup

def get_func_templ(code):
    url = 'http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=' + code
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "lxml")

    for s in soup.findAll('dt', {'class':'line-left'}):
        display( s )

display (get_func_templ('005930'))

<dt class=" line-left"><a href="../company/c1020001.aspx?cmp_cd=005930&amp;cn=">기업개요</a></dt>

<dt class=" line-left"><a href="../company/c1030001.aspx?cmp_cd=005930&amp;cn=">재무분석</a></dt>

<dt class=" line-left"><a href="../company/c1040001.aspx?cmp_cd=005930&amp;cn=">투자지표</a></dt>

<dt class=" line-left"><a href="../company/c1050001.aspx?cmp_cd=005930&amp;cn=">컨센서스</a></dt>

<dt class=" line-left"><a href="../company/c1060001.aspx?cmp_cd=005930&amp;cn=">업종분석</a></dt>

...

<dt class="line-left">PER <b class="num">17.36</b></dt>

<dt class="line-left">업종PER <b class="num">14.42</b></dt>

<dt class="line-left">PBR <b class="num">1.78</b></dt>

<dt class="line-left">현금배당수익률 <b class="num">1.20%</b></dt>

None

# 네이버 스크래핑 테스트

In [6]:
import requests
import time
from bs4 import BeautifulSoup

def get_sector(code):
    url = 'http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=' + code
    r = requests.get(url)
    soup = BeautifulSoup(r.text,"lxml")
    sector = ""

    # find KRX sector
    for s in soup.findAll('dt', {'class':'line-left'}):
        if s.text.find('KOSPI :') >= 0:
            sector = s.text.split(' : ')[1]
        elif s.text.find('KOSDAQ :') >= 0:
            sector = s.text.split(' : ')[1]

    return sector

display (get_sector('005930'))
display (get_sector('068270'))

'전기전자'

'제약'

# 포탈 종목 정보
* sector, wics, name_en
* desc, desc_date

In [7]:
import requests
import time
import json
from bs4 import BeautifulSoup

In [8]:
# homepage, phone, address
def get_naver_addres(code):
    homepage, phone, address = '', '', ''
    url = 'http://companyinfo.stock.naver.com/v1/company/c1020001.aspx?cmp_cd=' + code
    dfs = pd.read_html(url, encoding='utf-8')
    r = requests.get(url)
    soup = BeautifulSoup(r.text,"lxml")
    tab = soup.find('table', {'id':'cTB201'})
    if tab == None:
        return homepage, phone, address
    
    td = tab.find('td', {'class':'c2 txt'})
    homepage = td.a['href'].strip()

    td = tab.find('td', {'class':'c4 txt'})
    phone = td.text.strip()

    td = tab.find('td', {'class':'txt', 'colspan':'3'})
    address = td.text.strip()

    return homepage, phone, address


display( get_naver_addres('005930') )
display (get_naver_addres('105560'))

('http://www.samsung.com/sec',
 '031)200-1114      (주식담당: 02)2255-8126)',
 '경기도 수원시 영통구 삼성로 129(매탄동)')

('http://www.kbfg.com',
 '02)2073-7114      (주식담당: 02-2073-2843)',
 '서울특별시 중구 남대문로 84 (남대문로2가)')

In [9]:
# sector, wics, name_en
def get_sector(code):
    name_en, sector, wics, market = 'nan', 'nan', 'nan', 'nan'
    url = 'http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=' + code
    r = requests.get(url)
    soup = BeautifulSoup(r.text,"lxml")
    td = soup.find('td', {'class':'cmp-table-cell td0101'})
    if td == None:
        return sector, wics, name_en
        
    dts = td.findAll('dt')

    # dts[1], name_en
    name_en = dts[1].text
    
    # dts[2], sector
    s = dts[2]
    if s.text.find('KOSPI :') >= 0:
        market = 'KOSPI'
        sector = s.text.split(' : ')[1]
    elif dts[2].text.find('KOSDAQ :') >= 0:
        market = 'KOSDAQ'
        sector = s.text.split(' : ')[1]
            
    # dts[3], wics
    s = dts[3]
    if s.text.find('WICS :') >= 0:
        wics = s.text.split(' : ')[1]

    return market, wics, name_en


display( get_sector('005930') )
display (get_sector('105560'))


('KOSPI', '반도체와반도체장비', 'Samsung Electronics')

('KOSPI', '은행', 'KB Financial Group Inc.')

In [10]:
# desc, desc_date
def get_desc(code):
    url = 'http://companyinfo.stock.naver.com/v1/company/cmpcomment.aspx'
 
    cmt_text = ' '
    cmt_date = time.strftime("%Y-%m-%d")
 
    # preferred stock code to common stock
    if code[-1] in ['5', '7', '9']:
        code = code[0:-1] + '0'
 
    headers = {'Host':'companyinfo.stock.naver.com'}
    r = requests.post(url, data={'cmp_cd': code}, headers=headers)
    if r.text == "":
        return (cmt_text, cmt_date)
 
    j = json.loads(r.text)
    cmt_date = j['dt'].replace('.', '-')
    cmts = j['data'][0]
    cmts = [cmts['COMMENT_1'], cmts['COMMENT_2'], cmts['COMMENT_3'],  cmts['COMMENT_4'], cmts['COMMENT_5'] ]
    cmt_text = '. '.join(cmts)
    return (cmt_text, cmt_date)


display( get_desc('005930'))
display (get_desc('105560'))

('1969년 설립 된 글로벌 전자기업으로, 주요사업은 CE부문 (TV,냉장고 등)과 IM부문(스마트폰 등 HHP,네트워크시스템 등) 및 DS부문(DRAM,모바일AP,LCD패널 등) 3개의 부문으로 구성됨. 지역별로는 본사를 거점으로 한국 및 CE, IM부문 산하 해외 9개 지역총괄과 DS부문 산하 해외 5개 지역총괄의 생산,판매법인 등 169개의 동종업종을 영위하는 종속기업으로 구성되어 있음. IM부문은 프리미엄 브랜드 갤럭시를 필두로 보급형까지 풀라인업을 유지하며 글로벌 스마트폰 시장을 선도하고 있으며, Samsung Pay와 같은 Mobile Payment, Cloud 등 미래 성장 투자를 지속함. 반도체 사업은 DRAM, NAND Flash 제품 등 초미세 공정 기술을 적용하여 경쟁사 대비 차별성 있는 제품의 생산과 원가 경쟁력을 지속적으로 확보하여 전세계 메모리 시장의 선두 자리를 지속적으로 유지. 매출구성은 IM 40.3%, 반도체 20.5%, CE 18.9%, DP 10.8%, 기타 9.5% 등으로 구성',
 '2017-05-04')

('2008년 설립된 KB금융그룹의 지주회사로서 업계 선두권의 시장지위와 높은 브랜드 인지도를 바탕으로 은행을 비롯하여 카드, 증권, 생명보험, 손해보험, 저축은행 등 다양한 사업을 영위하고 있음. 연결대상 종속회사는 은행업(KB국민은행), 신용카드업(KB국민카드), 금융투자업(KB투자증권, KB자산운용, KB부동산신탁, KB인베스트먼트), 보험업(KB생명보험), 기타(KB신용정보, KB데이타시스템) 등이 있음. 자산 및 이익기여도 면에서 은행의 비중이 경쟁사 대비 높은 수준이며, 국민은행은 예수금 및 대출금 기준 시장점유율 1위의 시장지위를 바탕으로 대규모 이익을 창출하면서 그룹의 재무안정성에 큰 기여. 2013년 KB생명보험 인수를 시작으로,2014년 우리파이낸셜,2015년 LIG손해보험 인수하였고,2016년 4월 현대증권 지분에 대한 주식매매계약 체결하는 등 M＆A를 통한 비은행부문 경쟁력 강화하고 있음. 매출구성은 이자수익 39.5%, 유가증권평가및처분이익 27.6%, 외환거래이익 14.1%, 수수료수익 12.4%, 신탁업무운용수익 4.7%, 기타영업수익 1.1%, 배당금수익 0.5% 등으로 구성',
 '2017-04-23')

In [11]:
# 테이블  읽기
df_desc = pd.read_sql_query('SELECT * FROM stock_desc', conn)
df_desc.head()

,index,code,name,hompage,telephone,address,market,wics,name_en,desc,desc_date
0,0,060310,3S,,,,KOSDAQ,반도체와반도체장비,3S Korea,동사는 1991년 설립되어 2002년 코스닥시장에 상장. 반도체웨이퍼 캐리어박스와 ...,2017-04-23
1,1,095570,AJ네트웍스,,,,KOSPI,도로와철도운송,AJ Networks,2000년 2월 설립되었으며 2013년 12월 지배회사인 아주엘앤에프홀딩스(주)를 ...,2017-04-26
2,2,068400,AJ렌터카,,,,KOSPI,도로와철도운송,AJ Rent A Car,"동사는 1988년 브이아이피렌터카로 설립되어 자동차대여, 자동차운송 및 자동차정비사...",2017-04-23
3,3,006840,AK홀딩스,,,,KOSPI,화학,"AK Holdings, Inc.","동사는 2012년 9월 1일 인적분할을 통해, 신설회사 애경유화(주)(2012년 9...",2017-04-18
4,4,054620,APS홀딩스,,,,KOSDAQ,디스플레이장비및부품,APS Holdings Corporation,"1994년 9월 설립된 동사는 주요사업은 장비사업으로 AMOLED장비 제조, 반도체...",2017-04-22


In [12]:
# df_desc
# name, telephone, address, market, code, wics, name_en, desc, desc_date, corp_type, market_cap, update_date

cur = conn.cursor()

#iterows = df_desc.iterrows() # 전체
iterows = df_desc[:5].iterrows() # 처음 5개만
for idx, row in iterows:
    if row['wics'] == '':
        print('sector', row['code'])
        market, wics, name_en = get_sector(row['code'])
        cur.execute('UPDATE stock_desc SET market=?, wics=?, name_en=? WHERE code=?', (market, wics, name_en, row['code']) )
        conn.commit()

    if row['desc_date'] == '':
        print('desc', row['code'])
        desc, desc_date = get_desc(row['code'])
        cur.execute('UPDATE stock_desc SET desc=?, desc_date=? WHERE code=?', (desc, desc_date, row['code']) )
        conn.commit()


In [13]:
cursor = conn.execute("SELECT * from stock_desc limit 5")
for row in cursor:
    print (row)

(0, '060310', '3S', '', '', '', 'KOSDAQ', '반도체와반도체장비', '3S Korea', '동사는 1991년 설립되어 2002년 코스닥시장에 상장. 반도체웨이퍼 캐리어박스와 냉동공조용 열량계, 환경시험설비 전문업체임. 냉난방 능력의 측정 및 실내외 기기 조합의 적합성, 내구성, 실용성을 측정하는 냉동공조기 측정장치 ‘칼로리메타’를 생산해 공급. 동사는 칼로리메타의 국산화에 최초로 성공하여 삼성전자, LG전자, 위니아만도, 한라공조 등을 비롯한 국내외 공조기 제조업체, 대학교, 연구기관 등에 납품. 동사에서 제작 중인 칼로리메타의 종류로는 에어컨 칼로리메타, 전열교환기 칼로리메타, 자동차에어컨 칼로리메타, 자동차컴프레셔 칼로리메타, 압축기 시험장치 등이 있으며 국내 시장 점유율 약 60%를 차지. 매출구성은 칼로리메타 외 68.9%, 웨이퍼캐리어 30.8%, 냉동부품 0.3% 등으로 구성', '2017-04-23')
(1, '095570', 'AJ네트웍스', '', '', '', 'KOSPI', '도로와철도운송', 'AJ Networks', '2000년 2월 설립되었으며 2013년 12월 지배회사인 아주엘앤에프홀딩스(주)를 흡수 합병하였고, 자회사 등의 업무지원 사업을 영위하는 지주부문과 동사의 주요사업을 영위하는 사업부문으로 조직을 개편함. 당사는 파렛트, OA기기, 건설장비 장단기 대여, 판매업, 고소장비 사업을 영위. 렌탈 산업에 특화된 AJ그룹의 지주사로 AJ렌터카(자동차), AJ토탈(냉장/냉동창고), AJ파크(주차장) 등의 자회사를 보유함. 파레트영업망과 고소장비의 정비 능력의 시너지 효과를 증가시키기 위해 포크리프트렌탈 사업에 진출하였음. 매출구성은 렌탈매출 72.8%, 상품유통 23.7%, 기타 3.5%, 자산매각 0.02% 등으로 구성', '2017-04-26')
(2, '068400', 'AJ렌터카', '', '', '', 'KOSPI', '도로와철도운송', 'AJ Rent A Car', '동사는 1988년 브이아이

In [14]:
# 테이블  읽기
df = pd.read_sql_query('SELECT * FROM stock_desc', conn)
df.tail()

,index,code,name,hompage,telephone,address,market,wics,name_en,desc,desc_date
2139,2139,010240,흥국,,,,,,,,
2140,2140,189980,흥국에프엔비,,,,,,,,
2141,2141,000540,흥국화재,,,,,,,,
2142,2142,003280,흥아해운,,,,,,,,
2143,2143,037440,희림,,,,,,,,


# 전체 코드
* stock_desc.py

In [15]:
%%file stock_desc.py
#!/usr/bin/python

import pandas as pd
import requests
import time
import json
from bs4 import BeautifulSoup
import sqlite3

# sector, wics, name_en
def get_naver_sector(code):
    name_en, sector, wics, market = 'nan', 'nan', 'nan', 'nan'
    url = 'http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=' + code
    r = requests.get(url)
    soup = BeautifulSoup(r.text,"lxml")
    td = soup.find('td', {'class':'cmp-table-cell td0101'})
    if td == None:
        return sector, wics, name_en
        
    dts = td.findAll('dt')

    # dts[1], name_en
    name_en = dts[1].text
    
    # dts[2], sector
    s = dts[2]
    if s.text.find('KOSPI :') >= 0:
        market = 'KOSPI'
        sector = s.text.split(' : ')[1]
    elif dts[2].text.find('KOSDAQ :') >= 0:
        market = 'KOSDAQ'
        sector = s.text.split(' : ')[1]
            
    # dts[3], wics
    s = dts[3]
    if s.text.find('WICS :') >= 0:
        wics = s.text.split(' : ')[1]

    return market, wics, name_en

# desc, desc_date
def get_naver_desc(code):
    url = 'http://companyinfo.stock.naver.com/v1/company/cmpcomment.aspx'
 
    cmt_text = ' '
    cmt_date = time.strftime("%Y-%m-%d")
 
    if code[-1] == '5': # pre_order
        code = code[0:-1] + '0'
    if code[-1] == '7': # pre_order
        code = code[0:-1] + '0'
 
    headers = {'Host':'companyinfo.stock.naver.com'}
    r = requests.post(url, data={'cmp_cd': code}, headers=headers)
    if r.text == "":
        return (cmt_text, cmt_date)
 
    j = json.loads(r.text)
    cmt_date = j['dt'].replace('.', '-')
    cmts = j['data'][0]
    cmts = [cmts['COMMENT_1'], cmts['COMMENT_2'], cmts['COMMENT_3'],  cmts['COMMENT_4'], cmts['COMMENT_5'] ]
    cmt_text = '. '.join(cmts)
    return (cmt_text, cmt_date)

# homepage url, telephone, address
def get_naver_addres(code):
    homepage, telephone, address = '', '', ''
    url = 'http://companyinfo.stock.naver.com/v1/company/c1020001.aspx?cmp_cd=' + code
    r = requests.get(url)
    soup = BeautifulSoup(r.text,"lxml")
    tab = soup.find('table', {'id':'cTB201'})
    if tab == None:
        return homepage, telephone, address
    
    td = tab.find('td', {'class':'c2 txt'})
    if td.a:
        homepage = td.a['href'].strip()

    td = tab.find('td', {'class':'c4 txt'})
    telephone = td.text.strip()

    td = tab.find('td', {'class':'txt', 'colspan':'3'})
    address = td.text.strip()

    return homepage, telephone, address
    
# create stock_desc table
def create_stock_desc(conn):
    exist_sql = "SELECT name FROM sqlite_master WHERE type='table' AND name='stock_desc';"
    df_exist = pd.read_sql(exist_sql, conn)
    if len(df_exist) > 0:
        return False
    
    # create stock_desc
    df_master = pd.read_sql('SELECT * FROM stock_master', conn)
    df_desc = df_master[['code', 'name']]
    df_desc['hompage'] = ''
    df_desc['telephone'] = ''
    df_desc['address'] = ''
    df_desc['market'] = ''
    df_desc['wics'] = ''
    df_desc['name_en'] = ''
    df_desc['desc'] = ''
    df_desc['desc_date'] = ''
    df_desc.to_sql('stock_desc', conn)
    return True
        
if __name__ == "__main__":
    conn = sqlite3.connect('findata.db')
    create_stock_desc(conn)

    df_desc = pd.read_sql("SELECT * FROM stock_desc", conn)
    cur = conn.cursor()

    for idx, row in df_desc.iterrows():
        if row['homepage'] == '' or row['homepage'] == None:
            print('sector', row['code'], row['name'])
            homepage, telephone, address = get_naver_addres(row['code'])
            print(homepage, telephone, address)
            cur.execute('UPDATE stock_desc SET homepage=?, telephone=?, address=? WHERE code=?', 
                        (homepage, telephone, address, row['code']) )
            conn.commit()

        if row['wics'] == '':
            print('sector', row['code'], row['name'])
            market, wics, name_en = get_naver_sector(row['code'])
            print (market, wics, name_en)
            cur.execute('UPDATE stock_desc SET market=?, wics=?, name_en=? WHERE code=?', 
                        (market, wics, name_en, row['code']) )
            conn.commit()
    
        if row['desc_date'] == '':
            desc, desc_date = get_naver_desc(row['code'])
            print (desc, desc_date)
            cur.execute('UPDATE stock_desc SET desc=?, desc_date=? WHERE code=?', 
                        (desc, desc_date, row['code']) )
            conn.commit()
        print(row['code'], row['name'])
        
    conn.close()

Overwriting stock_desc.py


# 실행하기

명령어 라인에서 아래와 같이 실행합니다.

```bash
$ python3 stock_desc.py
```

----
#### 2017 FinanceData http://fb.com/financedata http://financedata.github.com